# Transfer Learning Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import sys
sys.path.append("..")
import math
import os 
import numpy as np
from torch.utils.data import Dataset,DataLoader,SubsetRandomSampler
from torchvision import transforms
import torchvision.transforms.functional as F
from models import *
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
%matplotlib inline

In [ ]:
net = ResNeXt29_2x64d()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = net.to(device)
net = torch.nn.DataParallel(net)
checkpoint = torch.load('../checkpoint/ckpt.t7')
net.load_state_dict(checkpoint['net'])
print("Weigths Loaded")

In [ ]:
net

In [ ]:
transform_train = transforms.Compose([ 
    transforms.ToPILImage(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),  
    transforms.RandomVerticalFlip(p=0.4),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [ ]:
X_train,y_train,X_test,y_test = d['X_train'], d['y_train'],d['X_test'], d['y_test']

In [ ]:
class CustomDataLoader(Dataset):
    
    def __init__(self, datax,datay,transform=None):
        self.datax = datax      
        self.datay = datay
        self.transform = transform
        
    def __len__(self):
        return len(self.datax)
    
    def __getitem__(self, index):
        image = self.datax[index]
        label = self.datay[index]
        
        if self.transform is not None:
            image = self.transform(image)
            
        return image, label

In [ ]:
train_loader = CustomDataLoader(X_train,y_train,transform=transform_train)
test_loader = CustomDataLoader(X_test,y_test,transform=transform_test)
data_model_train = DataLoader(train_loader,batch_size=bs,shuffle=True)
data_model_test = DataLoader(test_loader,batch_size=bs,shuffle=True)

In [ ]:
class Flatten(nn.Module):
    def forward(self, input):        
        return input.view(input.size(0), -1)
    
def model_head(n=3):
    m = list(net.children())[0]
    mHead = nn.Sequential(nn.AdaptiveAvgPool2d(2),
                          Flatten(),
                          nn.Linear(1024*2*2,512),
                          nn.BatchNorm1d(512),
                          nn.ReLU(inplace=True),
                          nn.Linear(512,128),
                          nn.BatchNorm1d(128),
                          nn.ReLU(inplace=True),
                          nn.Linear(128,n),
                          nn.Softmax(dim=1))
    m = nn.Sequential(*list(m.children())[:-1],
                      *list(mHead.children()));
    m = m.to(device)
    return m

In [ ]:
class MakeNetwork(nn.Module):
    def __init__(self,units):
        super(MakeNetwork,self).__init__()
        self.units = units
        self.model = model_head(units)   
    def forward(self,x):          
        return self.model(x)

In [ ]:
model = MakeNetworks([2]); model = model.to(device)

In [ ]:
paramNet = list(list(agns.children())[0].parameters())

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(paramNet,lr = 0.0001,momentum=0.9)

In [ ]:
def train(epoch,dm):
    print('\nEpoch: %d' % epoch)
    agns.train()   
    train_loss = 0
    correct = 0
    total = 0
    best_acc = 0
    for batch_idx, (inputs, targets) in enumerate(dm):
        inputs, targets = inputs.to(device), targets.to(device)        
        optimizer.zero_grad()  
        outputs = model(inputs)
        loss = criterion(outputs[0],targets)                     
        loss.backward()
        optimizer.step()  
        train_loss += loss.item()
        _, predicted = outputs[0].max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()  
        print('L: %.3f |Ac: %.3f%% .. (%.3f)|'% (train_loss/(batch_idx+1), 100.*correct/total,loss))    
